In [41]:
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
import sqlite3
import DataFunctions
import numpy as np


configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Data Preprocessing

Games DF

In [42]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"

# no longer need, right?
# try:
#     api_response = api_instance.get_games(year=year,division=division)
#     print(len(api_response))
# except:
#     print(ApiException)
    

In [43]:
gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=2022)
len(gamelist)

735

In [44]:
games_df = DataFunctions.df_from_games(gamelist=gamelist)
games_df.head()

,away_conference,away_id,away_points,away_team,home_conference,home_id,home_points,home_team,id,neutral_site,season
0,Big Ten,158,28,Nebraska,Big Ten,77,31,Northwestern,401405059,True,2022
1,Mountain West,2751,6,Wyoming,Big Ten,356,38,Illinois,401405058,False,2022
2,FBS Independents,41,20,Connecticut,Mountain West,328,31,Utah State,401404146,False,2022
3,Conference USA,2429,13,Charlotte,Conference USA,2226,43,Florida Atlantic,401426530,False,2022
4,Conference USA,249,31,North Texas,Conference USA,2638,13,UTEP,401426531,False,2022


In [45]:
conn = sqlite3.connect("CollegeFootball.db")
games_df.to_sql("games",conn,if_exists="replace",index=False)
conn.close()


In [6]:
# we can see with this code that we can extract DF from SQL db

# conn = sqlite3.connect("CollegeFootball.db")
# trialdf = pd.read_sql_query("SELECT * FROM games",conn)
# conn.close()
# print(trialdf.shape)
# trialdf.head()


Betting lines DF

In [7]:
api_instance1 = cfbd.BettingApi(cfbd.ApiClient(configuration))

In [8]:
conferences=[]
for game in gamelist:
    conferences.append(game.away_conference)
conferences=set(conferences)

conferences

{'ACC',
 'American Athletic',
 'Big 12',
 'Big Ten',
 'Conference USA',
 'FBS Independents',
 'Mid-American',
 'Mountain West',
 'Pac-12',
 'SEC',
 'Sun Belt'}

In [9]:
betting_list=DataFunctions.get_fbs_betting(api_instance=api_instance1,year=2022,conferences=conferences)

In [35]:
betting_df=DataFunctions.df_betting_lines(betting_list)
betting_df

#warning seems to be since betting line -0.0 for one of the games but still stores value as 0 so should be fine

/Users/daniel/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/daniel/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,id,av_spread,av_total
0,401411101,-24.375,51.375
1,401404079,7.125,65.625
2,401403893,1.500,51.000
3,401404080,2.500,45.000
4,401404081,13.625,53.000
...,...,...,...
733,401403935,12.500,50.500
734,401404029,21.125,61.125
735,401416641,-2.750,55.750
736,401404036,11.500,47.000


In [46]:
conn=sqlite3.connect("CollegeFootball.db")
betting_df.to_sql("betting_lines",conn,if_exists="replace",index=False)
conn.close()

In [47]:
#sample merge dataframe

cmd=\
"""
SELECT G.away_team,G.away_points,G.home_points,G.home_team,B.av_spread,B.av_total
FROM games G
INNER JOIN betting_lines B ON G.id=B.id
"""
conn=sqlite3.connect("CollegeFootball.db")
test_merge=pd.read_sql_query(cmd,conn)
test_merge

,away_team,away_points,home_points,home_team,av_spread,av_total
0,Nebraska,28,31,Northwestern,-12.375,52.250
1,Wyoming,6,38,Illinois,14.000,42.750
2,Connecticut,20,31,Utah State,23.750,59.250
3,Charlotte,13,43,Florida Atlantic,7.000,60.000
4,North Texas,31,13,UTEP,-1.375,52.875
...,...,...,...,...,...,...
730,UCF,28,45,Tulane,3.500,57.375
731,Fresno State,28,16,Boise State,3.000,54.750
732,Purdue,22,43,Michigan,15.625,53.750
733,Clemson,39,10,North Carolina,-7.250,64.000


Team stats per season DF

Merging tables to set up for NN